In [1]:
import pandas as pd
from gensim import utils
import json
import csv
from smart_open import open
from hashlib import sha1
from datasketch import MinHash, MinHashLSH
from tqdm import tqdm
tqdm.pandas()

In [23]:
from spacy.lang.ru import Russian
nlp = Russian()
nlp.add_pipe('sentencizer')

In [2]:
def process(section):
    res = []
    doc = nlp(section)
    for sent in doc.sents:
        sent = str(sent)
        x = sent.replace('\n', '')
        if x:
            res.append([x])
    return res

In [15]:
%%time

i = 0

with open('ruwiki-latest.json.gz', 'rb') as f:
        for line in f:
            i += 1

print(f'Total articles  -- {i}')

Total articles  -- 2801969
Wall time: 1min 25s


In [20]:
# %%time

# import csv
# i = 0
# cnt = 0
# file_name = 'texts0.csv'
# cs = open(file_name, 'w', newline='', encoding='utf-8')
# linewriter = csv.writer(cs)

# tmp = []

# with open('ruwiki-latest.json.gz', 'rb') as f:

#         for line in f:
#             i += 1
#             article = json.loads(line)

#             for section_text in article['section_texts']:
#                 art = process(section_text)
#                 tmp.extend(art)
#                 tmp.append(['EOS'])        

#             if i == 466995:
#                 linewriter.writerows(tmp)
#                 tmp = []
#                 cs.close() 
#                 i = 0          
#                 cnt += 1
#                 print('Starting new file')
#                 file_name = 'texts' + str(cnt) + '.csv'
#                 cs = open(file_name, 'w', newline='', encoding='utf-8')
#                 linewriter = csv.writer(cs)
            
#             elif i % 100000 == 0:
#                 print(f'Current i -- {i}')
#             if len(tmp) > 100:
#                 linewriter.writerows(tmp)
#                 tmp = []

# if tmp:
#     linewriter.writerows(tmp) 
#     tmp = []
#     cs.close()

Current i -- 100000
Current i -- 200000
Current i -- 300000
Current i -- 400000
Starting new file
Current i -- 0
Current i -- 100000
Current i -- 200000
Current i -- 300000
Current i -- 400000
Starting new file
Current i -- 0
Current i -- 100000
Current i -- 200000
Current i -- 300000
Current i -- 400000
Starting new file
Current i -- 0
Current i -- 100000
Current i -- 200000
Current i -- 300000
Current i -- 400000
Starting new file
Current i -- 0
Current i -- 100000
Current i -- 200000
Current i -- 300000
Current i -- 400000
Starting new file
Current i -- 0
Current i -- 100000
Current i -- 200000
Current i -- 300000
Current i -- 400000
Wall time: 1h 6min 53s


In [2]:
def hashes_simple(x):
    m1 = MinHash(num_perm=16)
    x = x.lower()
    x = set(x.split())
    _ = [m1.update(token.encode('utf8')) for token in x]
    return m1

In [3]:
def process_txt(x):
    if x == '\n':
        return x
    else:
        x = x.replace('\n', '')
        return x

In [6]:
df = pd.read_csv('texts0.csv', header=None, names = ['Text'],
                             encoding='utf-8', low_memory=True, nrows=50)

# Upload hashes in memory (csv)

In [5]:
%%time

lsh = MinHashLSH(threshold=0.8, num_perm=16)

comp = 1000000
i = 0
with lsh.insertion_session() as session:
    for chunk in pd.read_csv('texts5.csv', header=None, names = ['Text'],
                             encoding='utf-8', low_memory=True, chunksize=500000):
        chunk = chunk.dropna()
        chunk = chunk[chunk['Text'] != 'EOS']
        texts = chunk.Text.progress_apply(hashes_simple)
        texts = texts.to_list()
        hashes = [('m'+str(u), x) for x, u in zip(texts, range(i, i+len(texts)+1))]
        i += len(texts)
        if i > comp:
            print(f'Processed sentences -- {i:,}')
            comp += 1000000
        for key, minhash in hashes:
            session.insert(key, minhash)

In [4]:
print('All hashes are added to the LSH')

All hashes are added to the LSH


In [5]:
print(f'Total number of sentences in file -- {i:,}')

Total number of sentences in file -- 6,449,430


In [6]:
del texts
del chunk
del hashes
del i
del comp

# Upload hashed into memory (txt)

In [6]:
%%time

lsh = MinHashLSH(threshold=0.8, num_perm=16)

comp = 1000000
i = 0

def read_in_chunks_lines(file_object, chunk_size=40000000):
    while True:
        data = file_object.readlines(chunk_size)
        if not data:
            break
        yield data

with lsh.insertion_session() as session:
    with open('rnc_post-soviet_cleaned.txt', encoding='utf-8') as f1:
        for piece in read_in_chunks_lines(f1):
            sents = [process_txt(x) for x in piece]
            texts = [hashes_simple(x) for x in tqdm(sents)]
            hashes = [('m'+str(u), x) for x, u in zip(texts, range(i, i+len(texts)+1))]
            i += len(texts)
            if i > comp:
                print(f'Processed sentences -- {i:,}')
                comp += 1000000
            for key, minhash in hashes:
                session.insert(key, minhash)

100%|████████████████████████████████████████████████████████████████████████| 446876/446876 [02:55<00:00, 2542.26it/s]


Processed sentences -- 1,341,707


100%|████████████████████████████████████████████████████████████████████████| 448422/448422 [03:01<00:00, 2467.95it/s]


Processed sentences -- 2,237,772


100%|████████████████████████████████████████████████████████████████████████| 449315/449315 [03:03<00:00, 2452.78it/s]


...

Processed sentences -- 5,373,185


100%|████████████████████████████████████████████████████████████████████████| 447795/447795 [03:23<00:00, 2205.35it/s]


Processed sentences -- 6,268,278


100%|████████████████████████████████████████████████████████████████████████| 267312/267312 [01:50<00:00, 2409.40it/s]


Wall time: 45min 50s


In [7]:
print('All hashes are added to the LSH')

All hashes are added to the LSH


In [8]:
print(f'Total number of sentences in file -- {i:,}')

Total number of sentences in file -- 6,535,590


In [9]:
del texts
del piece
del hashes
del i
del comp

# csv cleaning

In [39]:
%%time

d = 0
with open('geo_wac8_v_01.txt', 'w', encoding='utf-8') as f:
    for chunk in pd.read_csv('geo_wac8.csv', header=None, names = ['Text'], encoding='utf-8',
                             low_memory=True, chunksize=100000):
        df_geo = chunk.Text.to_list()
        geo_sents = []

        for text in df_geo:
            sents = process(text)
            geo_sents.extend(sents)
            geo_sents.append(['\n'])

        geo_sents = [item for sublist in geo_sents for item in sublist]
        geo_hash = [hashes_simple(x) for x in geo_sents]

        for i in tqdm(range(len(geo_hash))):
            key = geo_hash[i]
            if geo_sents[i] == '\n':
                f.write('\n')
            elif len(lsh.query(key)) == 0:
                f.write(geo_sents[i] + '\n')
            else:
                d += 1
                if d % 100000 == 0:
                    print(f'Duplicates found --- {d:,}')

Duplicates found --- 100,000




Wall time: 1h 20min 56s


In [40]:
print(f'Total duplicates in file --- {d}')

Total duplicates in file --- 175647


# txt cleaning

In [18]:
%%time

d = 0

def read_in_chunks_lines(file_object, chunk_size=40000000):
    while True:
        data = file_object.readlines(chunk_size)
        if not data:
            break
        yield data

with open('geo_wac6_v_09.txt', 'w', encoding='utf-8') as f:
    with open('geo_wac6_v_08_cleaned.txt', encoding='utf-8') as f1:
        for piece in read_in_chunks_lines(f1):
            geo_sents = [process_txt(x) for x in piece]
            geo_hash = [hashes_simple(x) for x in tqdm(geo_sents)]
            for i in range(len(geo_hash)):
                key = geo_hash[i]
                if geo_sents[i] == '\n':
                    f.write('\n')
                elif len(lsh.query(key)) == 0:
                    f.write(geo_sents[i] + '\n')
                else:
                    d += 1
                    if d % 10000 == 0:
                        print(f'Duplicates found --- {d:,}')

100%|████████████████████████████████████████████████████████████████████████| 526124/526124 [03:44<00:00, 2339.47it/s]


Duplicates found --- 10,000


100%|████████████████████████████████████████████████████████████████████████| 491562/491562 [03:28<00:00, 2354.25it/s]


Duplicates found --- 20,000


100%|████████████████████████████████████████████████████████████████████████| 449122/449122 [03:16<00:00, 2291.32it/s]


...

Duplicates found --- 40,000


100%|████████████████████████████████████████████████████████████████████████| 489072/489072 [03:21<00:00, 2424.70it/s]


Duplicates found --- 50,000


100%|██████████████████████████████████████████████████████████████████████████| 10353/10353 [00:03<00:00, 2778.71it/s]


Wall time: 2h 31min 28s


In [19]:
print(f'Total duplicates in the file -- {d:,}')

Total duplicates in the file -- 58,871


# Compute the number of words without stop-words with and without punctuation

Total: 914,576,929 tokens with punctuation but without stop-words 

Total: 679,042,226 tokens without punctuation and without stop-words

In [5]:
import time
import pytz
from datetime import datetime

In [1]:
from spacy.lang.ru import Russian
nlp = Russian()
tokenizer = nlp.tokenizer

In [18]:
%%time

cnt_tok_punc = 0
cnt_tok_wo_punc = 0
i = 10000000

def read_in_chunks(file_object, chunk_size=5000):
    """Lazy function (generator) to read a file piece by piece.
    Default chunk size: 1k."""
    while True:
        data = file_object.read(chunk_size)
        if not data:
            break
        yield data


with open('text.txt', encoding='utf-8') as f:
    for piece in read_in_chunks(f):
      doc = tokenizer(piece)
      cnt_tok_punc += len([token.text for token in doc if not token.is_stop])
      cnt_tok_wo_punc += len([token for token in doc if not token.is_stop and not token.is_punct])
      if cnt_tok_punc > i:
        datetime_M = datetime.now().strftime("%H:%M:%S")
        print(f'Current {cnt_tok_punc} tokens (w/o stops) with punctuation, time {datetime_M}')
        print(f'Current {cnt_tok_wo_punc} tokens (w/o stops) without punctuation, time {datetime_M}')  
        print('\n')
        i += 10000000

print(f'Total: {cnt_tok_punc} tokens (w/o stops) with punctuation')
print(f'Total: {cnt_tok_wo_punc} tokens (w/o stops) without punctuation')  

Current 10000184 tokens (w/o stops) with punctuation, time 14:26:53
Current 7367015 tokens (w/o stops) without punctuation, time 14:26:53


Current 20000246 tokens (w/o stops) with punctuation, time 14:27:58
Current 14848526 tokens (w/o stops) without punctuation, time 14:27:58


Current 30000054 tokens (w/o stops) with punctuation, time 14:28:56
Current 22313159 tokens (w/o stops) without punctuation, time 14:28:56


Current 40000743 tokens (w/o stops) with punctuation, time 14:30:04
Current 29727063 tokens (w/o stops) without punctuation, time 14:30:04


Current 50000693 tokens (w/o stops) with punctuation, time 14:31:08
Current 37137184 tokens (w/o stops) without punctuation, time 14:31:08


Current 60000513 tokens (w/o stops) with punctuation, time 14:32:08
Current 44512651 tokens (w/o stops) without punctuation, time 14:32:08


Current 70000593 tokens (w/o stops) with punctuation, time 14:33:02
Current 51952056 tokens (w/o stops) without punctuation, time 14:33:02


Current 800006

# Compute the number of words with and without punctuation


Total: 1,044,197,175 tokens with punctuation

Total: 808,662,472 tokens without punctuation

In [8]:
%%time

cnt_tok_punc = 0
cnt_tok_wo_punc = 0
i = 1000000

def read_in_chunks(file_object, chunk_size=5000):
    """Lazy function (generator) to read a file piece by piece.
    Default chunk size: 1k."""
    while True:
        data = file_object.read(chunk_size)
        if not data:
            break
        yield data


with open('text.txt', encoding='utf-8') as f:
    for piece in read_in_chunks(f):
      doc = tokenizer(piece)
      cnt_tok_punc += len([w.text for w in doc])
      cnt_tok_wo_punc += len([token for token in doc if not token.is_punct])
      if cnt_tok_punc > i:
        datetime_M = datetime.now().strftime("%H:%M:%S")
        print(f'Current {cnt_tok_punc} tokens with punctuation, time {datetime_M}')
        print(f'Current {cnt_tok_wo_punc} tokens without punctuation, time {datetime_M}')  
        print('\n')
        i += 1000000

print(f'Total: {cnt_tok_punc} tokens with punctuation')
print(f'Total: {cnt_tok_wo_punc} tokens without punctuation')  

ent 926000475 tokens with punctuation, time 2021-04-16 20:56:11.580188+03:00
Current 716505997 tokens without punctuation, time 2021-04-16 20:56:11.580188+03:00


Current 927000015 tokens with punctuation, time 2021-04-16 20:56:14.130295+03:00
Current 717283578 tokens without punctuation, time 2021-04-16 20:56:14.130295+03:00


Current 928000571 tokens with punctuation, time 2021-04-16 20:56:16.709778+03:00
Current 718066381 tokens without punctuation, time 2021-04-16 20:56:16.709778+03:00


Current 929000307 tokens with punctuation, time 2021-04-16 20:56:19.310778+03:00
Current 718839303 tokens without punctuation, time 2021-04-16 20:56:19.310778+03:00


Current 930000602 tokens with punctuation, time 2021-04-16 20:56:21.883807+03:00
Current 719618600 tokens without punctuation, time 2021-04-16 20:56:21.883807+03:00


Current 931000446 tokens with punctuation, time 2021-04-16 20:56:24.444839+03:00
Current 720409414 tokens without punctuation, time 2021-04-16 20:56:24.444839+03:00


Cu